# Advent of Code - 2025 - Day 10 - Problem 1

https://adventofcode.com/2025/day/10

## Load Source Data

Load source data into `DATA`.

In [1]:
# Read the input file containing turn instructions
with open("data/day10.txt") as f:
    DATA = [line.strip() for line in f]

# DATA

## Define Machine class

Defines the state of a machine.

In [2]:
import itertools
import re


class Machine:

    def __init__(self, definition: str):

        valid_lights, buttons, joltages = Machine.parse_definition(definition)

        self.valid_lights = valid_lights
        self.buttons = buttons
        self.joltages = joltages

    @staticmethod
    def create_patterns() -> tuple[str, str]:

        ws = r"\s+"
        light_pattern = r"\[([.#]+)\]"
        button_pattern = r"\(([0-9,]+)\)"
        joltage_pattern = r"\{([0-9,]+)\}"

        # Regular expression pattern for the definition string
        line_pattern = (
            f"{light_pattern}((?:{ws}{button_pattern})+){ws}{joltage_pattern}"
        )

        # Regular expression for individual button definitions
        button_pattern = f"(?:{ws}({button_pattern}))"

        return (line_pattern, button_pattern)

    @staticmethod
    def parse_definition(
        definition: str,
    ) -> tuple[list[bool], list[list[int]], list[int]]:

        # Parse the overall definition string. Note that there are two results returned for the buttons due to the nested grouping constructs: the entire
        # button string as well as the last matched button.
        #
        definition_match = re.match(Machine.line_pattern, definition)
        assert definition_match
        buttons = definition_match.group(2)
        button_matches = re.findall(Machine.button_pattern, buttons)

        # Parse and convert the separate machine attributes.
        #
        valid_lights: list[bool] = [light == "#" for light in definition_match.group(1)]
        buttons = [
            list(map(int, button_match[1].split(",")))
            for button_match in button_matches
        ]
        joltages = list(map(int, definition_match.group(4).split(",")))

        return (valid_lights, buttons, joltages)

    def get_lights(self, button_presses: list[bool]) -> list[bool]:
        result: list[bool] = [False] * len(self.valid_lights)
        for idx_button, button_pressed in enumerate(button_presses):
            if button_pressed:
                button = self.buttons[idx_button]
                for idx_light in button:
                    result[idx_light] = not result[idx_light]

        return result

    def get_all_solutions(self) -> list[list[bool]]:

        solutions: list[list[bool]] = []

        for test_button_combination in itertools.product(
            [True, False], repeat=len(self.buttons)
        ):
            test_lights = self.get_lights(list(test_button_combination))
            if test_lights == self.valid_lights:
                solutions.append(list(test_button_combination))

        return solutions

    def get_best_solutions(self) -> list[bool]:

        all_solutions = self.get_all_solutions()
        all_solutions.sort(key=sum)

        best_solution = all_solutions[0]
        return best_solution

    line_pattern, button_pattern = create_patterns()

## Solve All Machines

In [3]:
total_presses = 0

for line in DATA:
    m = Machine(line)
    total_presses += sum(m.get_best_solutions())

print(f"total_presses = {total_presses}")

total_presses = 535
